In [50]:
get_ipython().magic('reset -sf')

# Models

In [51]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import neighbors
from sklearn import feature_selection
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import xgboost as xgb
import time
import dalex as dx
from sklearn.ensemble import RandomForestRegressor
import torch
import torch.nn as nn
from torch.autograd import Variable 
from captum.attr import (
    IntegratedGradients,
    GradientShap,
    DeepLift,
    DeepLiftShap,
    LayerConductance,
    NeuronConductance,
    NoiseTunnel,
)
import lime.lime_tabular
import pickle
import matplotlib.pyplot as plt
import sys
from torch.utils.data import DataLoader
from sklearn.ensemble import GradientBoostingRegressor

First, I import pre-cleaned data, after I divide the dataset in training and testing, 60% and 40% respectively. Everything has to be scaled and I prepare the folds for the 5 cross validation. The steps I am going to follow in wach model are the following:
- Recursive feature elimination, I am going to remove the non important features
- Grid search, in order to try all the hyperparameters and choose the best ones
- Create model with the hyperparameters selected previously and fit it
- Make the predictions
- Shapley values to understand the model and the predictions

In [52]:
data_info_original = pd.read_csv('info_data.csv')
data_info_original = data_info_original.drop(columns = ['DAY_OF_WEEK'])
data_info_original = data_info_original.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])
data_info = pd.read_csv('info_datav2.csv')
data_info = data_info.drop(columns = ['DAY_OF_WEEK'])
data_info = data_info.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])
data_gaussian = data_info[data_info.ANO_FACTURA == 2020]

In [53]:
print(data_info.shape)
print(data_info_original.shape)

In [54]:
def groupData(df, col_groups, ref_col):
    testing =  df.groupby(col_groups)[ref_col].sum()
    grouped_testing = []
    for k,v in zip(testing.index, testing.values):
        val = [k[0],k[1],k[2],v,k[3], k[4],k[5],k[6],k[7],k[8],k[9],k[10],k[11],k[12],k[13],k[14],k[15],k[16],k[17],k[18],k[19],k[20],k[21],k[22],k[23],k[24], k[25]]
        grouped_testing.append(val)
    grouped_testing = pd.DataFrame(grouped_testing, columns = df.columns)
    #grouped_testing = grouped_testing.reset_index(drop=True)
    return grouped_testing

def getISOCountry(iso_country):
    indexes = data_info_original[data_info_original['NUMERO_DEUDOR_PAIS_ID'] == iso_country].index
    iso_code = data_info.iloc[[indexes[0]]]['NUMERO_DEUDOR_PAIS_ID']
    return iso_code.values[0]

def getEncode(dto, dt, val, col):
    indexes = dto[-dto[col] == val].index
    code = dt.iloc[[indexes[0]]][col]
    return code.values[0]

def getValuesFilter(ds, value, columns, target):
    if value == '*': #no filter 
        return ds
    indexes = getIndexFilter(ds, value, target)
    datat = getValues(ds, indexes, columns)
    return datat

def getIndexFilter(dt, value, target):
    indexes = dt[dt[target] == value].index
    return indexes

def getValues(dt, indexes, columns):
    datat = []
    for k, v in zip(dt.index, dt.values):
        if k in indexes:
            datat.append(v)
    df = pd.DataFrame(datat, columns = columns)
    return df

def managementDataFile(dat, name_file, r_w):      
    # Its important to use binary mode
    if r_w == 'r':
        dbfile = open(name_file, 'ab')
        pickle.dump(dat, dbfile)
    elif r_w == 'w':
        dbfile = open(name_file, 'rb')
        db = pickle.load(dbfile)
    dbfile.close()

In [55]:
columns = ['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA', 'TEMPORADA_COMERCIAL_ID', 'PRODUCTO_ID', 'TALLA', 'ESFUERZO_VENTA_ID', 'NUMERO_DEUDOR_PAIS_ID', 'JERARQUIA_PROD_ID', 'GRUPO_ARTICULO_PRODUCTO_ID', 'GENERO_PRODUCTO', 'CATEGORIA', 'TIPOLOGIA', 'CONSUMER_COLOR', 'CREMALLERA', 'CORDONES', 'OUTSOLE_SUELA_TIPO', 'OUTSOLE_SUELA_SUBTIPO', 'PLANTILLA_EXTRAIBLE', 'CONTACTO_SN', 'EDAD_SN', 'GENERO_CONTACTO', 'EDAD_COMPRA', 'EDAD_RANGO_COMPRA', 'CIUDAD_CONTACTO', 'IDIOMA_CONTACTO']

In [56]:
data_info.columns

In [57]:
data_info_grouped = groupData(data_info, columns, 'IMP_VENTA_NETO_EUR')
data_info_grouped = data_info_grouped.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])
data_info_original_grouped = groupData(data_info_original, columns, 'IMP_VENTA_NETO_EUR')
data_info_original_grouped = data_info_original_grouped.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])

In [58]:
print(data_info_grouped.shape)
print(data_info_original_grouped.shape)

In [59]:
data_info_filtered_grouped = getValuesFilter(data_info_grouped, '*', data_info_grouped.columns, 'NUMERO_DEUDOR_PAIS_ID') # iso = * -> no filter by country
data_info_filtered_grouped = data_info_filtered_grouped.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])
data_info_original_filtered_grouped = getValuesFilter(data_info_original_grouped, '*', data_info_original_grouped.columns, 'NUMERO_DEUDOR_PAIS_ID') # iso = * -> no filter by country
data_info_original_filtered_grouped = data_info_original_filtered_grouped.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])

In [60]:
traindataset_original, testdataset_original = train_test_split(data_info_original_filtered_grouped, test_size=0.4, shuffle= False) # To use all the data, change to -> data_info
traindataset, testdataset = train_test_split(data_info_filtered_grouped, test_size=0.4, shuffle= False) # To use all the data, change to -> data_info
x_train = traindataset.loc[:, traindataset.columns != 'IMP_VENTA_NETO_EUR']
y_train = traindataset.loc[:, traindataset.columns == 'IMP_VENTA_NETO_EUR']
x_train = x_train.drop(columns=['EDAD_RANGO_COMPRA'])
x_test = testdataset.loc[:, testdataset.columns != 'IMP_VENTA_NETO_EUR']
y_test = testdataset.loc[:, testdataset.columns == 'IMP_VENTA_NETO_EUR']
x_test = x_test.drop(columns = 'EDAD_RANGO_COMPRA')

In [61]:
normalizer = MinMaxScaler(feature_range = (-1, 1))
x_train = pd.DataFrame(normalizer.fit_transform(x_train), columns= x_train.columns, index = traindataset.index)
x_test = pd.DataFrame(normalizer.fit_transform(x_test), columns= x_test.columns, index = testdataset.index)
mseresults = pd.DataFrame()
timeexecution = pd.DataFrame()

In [62]:
folds = KFold(n_splits = 5, shuffle = False) # if shuffle false, random state doesn't matter

## Linear Model

I have to choose which are the most important features in the model, I will use R2 to get it.

In [95]:
hyper_params = [{'n_features_to_select': list(range(1, 26))}]
lm = LinearRegression()
rfe = RFE(lm)
model_cv = GridSearchCV(estimator = rfe, param_grid = hyper_params, scoring= 'r2', cv = folds, verbose = 1, return_train_score=True)      
model_cv.fit(x_train, y_train)
cv_results = pd.DataFrame(model_cv.cv_results_)

In [ ]:
display(cv_results)
print(model_cv.best_params_)

In [ ]:
#plt.figure(figsize=(16,6))
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])
plt.xlabel('number of features')
plt.ylabel('r-squared')
plt.title("Optimal Number of Features")
plt.legend(['test score', 'train score'], loc='lower right')
plt.savefig('../Output/testscoretrain.png')
plt.show()

display(cv_results[['param_n_features_to_select', 'mean_train_score', 'mean_test_score']])

In [63]:
n_features_optimal = 25 # model_cv.best_params_['n_features_to_select']
lm = LinearRegression()
rfe = RFE(lm, n_features_to_select= n_features_optimal)             
rfe = rfe.fit(x_train, y_train)

I select only the parameters I am interested in

In [64]:
def removeFeatures(dt_train, dt_test, rfe_model):
    chosen = pd.DataFrame(rfe_model.support_, index=dt_train.columns, columns=['Rank'])
    featuresnotselected = []
    for k, v in zip(chosen.index, chosen.values):
        if v == False:
            featuresnotselected.append(k)
    dt_train_final = dt_train.drop(columns= featuresnotselected)
    dt_test_final = dt_test.drop(columns= featuresnotselected)
    return dt_train_final, dt_test_final

In [65]:
x_train_LM, x_test_LM = removeFeatures(x_train, x_test, rfe)

Finally, I create the model with the best parametrization possible

In [66]:
lm_model = LinearRegression()
start_time = time.time()
lm_model.fit(x_train_LM, y_train)
timeexecution['lm'] = (time.time() - start_time)
y_pred = lm_model.predict(x_test_LM)
results = pd.DataFrame(index= testdataset.index, columns = ['Import'])
results['Import'] = y_test.copy()
results['lm'] = y_pred

In [63]:
display(results)

#De moment no ho utilitzar�
F-test, p-values:

In [18]:
f_val, p_val = feature_selection.f_regression(x_train_LM, y_train) #Repassar
display(list(zip(x_train_LM.columns, p_val)))
print('')
display(list(zip(x_train_LM.columns, f_val)))

In [140]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2: ', metrics.r2_score(y_test, y_pred))
mseresults['lm'] = metrics.mean_squared_error(y_test, y_pred)

In [20]:
plt.barh(range(0, 25), lm_model.coef_[0])
plt.yticks(range(0, 25), x_train_LM.columns, fontsize = 8)
plt.title('Coefficients')
plt.savefig('../Output/coefflinear.png')
plt.show()

In [20]:
variable_groups = {
    'Date': ['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA', 'TEMPORADA_COMERCIAL_ID'],
    'Product': ['PRODUCTO_ID', 'TALLA', 'GRUPO_ARTICULO_PRODUCTO_ID', 'GENERO_PRODUCTO', 'CATEGORIA', 'TIPOLOGIA', 'CONSUMER_COLOR', 'CREMALLERA', 'CORDONES', 'OUTSOLE_SUELA_TIPO', 'OUTSOLE_SUELA_SUBTIPO', 'PLANTILLA_EXTRAIBLE'],
    'Age': ['EDAD_SN', 'EDAD_COMPRA'],
    'ContanctInfo': ['NUMERO_DEUDOR_PAIS_ID', 'CIUDAD_CONTACTO', 'IDIOMA_CONTACTO', 'GENERO_CONTACTO', 'CONTACTO_SN'],
    'SalePerson': ['ESFUERZO_VENTA_ID']
}

In [22]:
linear_explainer = dx.Explainer(lm_model, x_test_LM, y_test)
explanation = linear_explainer.model_parts()
explanation.plot()
groupedexpl = linear_explainer.model_parts(variable_groups=variable_groups)
groupedexpl.plot()

In [21]:
indexspaindata = getIndexFilter(testdataset, 14, 'NUMERO_DEUDOR_PAIS_ID')
indexspaindata_original = getIndexFilter(testdataset_original, 'ES', 'NUMERO_DEUDOR_PAIS_ID')
usa_iso = getISOCountry('US')
indexusadata = getIndexFilter(testdataset, usa_iso, 'NUMERO_DEUDOR_PAIS_ID')
indexusadata_original = getIndexFilter(testdataset_original, 'US', 'NUMERO_DEUDOR_PAIS_ID')

In [24]:
indexspaindata[0]

Spain data

In [27]:
display(testdataset_original.loc[indexspaindata_original[0]])
display(testdataset_original.loc[indexspaindata_original[20]])
display(testdataset_original.loc[indexspaindata_original[200]])
display(testdataset_original.loc[indexspaindata_original[500]])

In [28]:
bd_henry_es_0 = linear_explainer.predict_parts(np.ravel(x_test_LM.loc[[indexspaindata[0]]]), type = 'shap', B= 100)
bd_henry_es_0.plot()
bd_henry_es_1 = linear_explainer.predict_parts(np.ravel(x_test_LM.loc[[indexspaindata[20]]]), type = 'shap', B= 100)
bd_henry_es_1.plot()
bd_henry_es_2 = linear_explainer.predict_parts(np.ravel(x_test_LM.loc[[indexspaindata[200]]]), type = 'shap', B= 100)
bd_henry_es_2.plot()
bd_henry_es_3 = linear_explainer.predict_parts(np.ravel(x_test_LM.loc[[indexspaindata[500]]]), type = 'shap', B= 100)
bd_henry_es_3.plot()

US data

In [29]:
display(testdataset_original.loc[indexusadata_original[0]])
display(testdataset_original.loc[indexusadata_original[20]])
display(testdataset_original.loc[indexusadata_original[200]])
display(testdataset_original.loc[indexusadata_original[500]])

In [30]:
bd_henry_us_1 = linear_explainer.predict_parts(np.ravel(x_test_LM.loc[[indexusadata[0]]]), type = 'shap', B= 100)
bd_henry_us_1.plot()
bd_henry_us_2 = linear_explainer.predict_parts(np.ravel(x_test_LM.loc[[indexusadata[20]]]), type = 'shap', B= 100)
bd_henry_us_2.plot()
bd_henry_us_3 = linear_explainer.predict_parts(np.ravel(x_test_LM.loc[[indexusadata[200]]]), type = 'shap', B= 100)
bd_henry_us_3.plot()
bd_henry_us_4 = linear_explainer.predict_parts(np.ravel(x_test_LM.loc[[indexusadata[500]]]), type = 'shap', B= 100)
bd_henry_us_4.plot()

## XGBoost Model

For this model, I am going to work with, the parameters max_depth, learning_rate and subsample from the Xgboost model and n_features_to_select from RFE.

In [ ]:
dtrain = xgb.DMatrix(x_train, label=y_train, feature_names=list(x_train.columns))
dtest = xgb.DMatrix(x_test, feature_names=list(x_test.columns))

In [18]:
#hyper_params = {'estimator__max_depth':[1, 10, 15], 'estimator__n_estimators': [150, 300, 500], 'estimator__learning_rate':[0.1, 0.05, 0.01] }#, 'n_features_to_select': list(range(1, 26))}
hyper_params = {'n_features_to_select': list(range(16, 26))}
xgbm = xgb.XGBRegressor(learning_rate =0.01, n_estimators=215, max_depth=10, min_child_weight=0.8, subsample=1, nthread=4)
rfe_xgboost = RFE(xgbm)
model_cv_xgb = GridSearchCV(estimator = rfe_xgboost, param_grid = hyper_params, scoring= 'r2', cv = folds, verbose = 1, return_train_score=True)      
model_cv_xgb.fit(x_train, y_train)
cv_results_xgb = pd.DataFrame(model_cv_xgb.cv_results_)

In [ ]:
display(cv_results_xgb)
print(model_cv_xgb.best_params_)

In [ ]:
#plt.figure(figsize=(16,6))
plt.plot(cv_results_xgb["param_n_features_to_select"], cv_results_xgb["mean_test_score"])
plt.plot(cv_results_xgb["param_n_features_to_select"], cv_results_xgb["mean_train_score"])
plt.xlabel('number of features')
plt.ylabel('r-squared')
plt.title("Optimal Number of Features")
plt.legend(['test score', 'train score'], loc='lower right')
plt.savefig('../Output/testscoretrain_xgb.png')
plt.show()

display(cv_results_xgb[['param_n_features_to_select', 'mean_train_score', 'mean_test_score']])

Once I get the best parametrization, I execute the model with it, by the moment this is an example because I haven't execute the previous, too long.

In [17]:
n_features_optimal = 21 # model_cv.best_params_['n_features_to_select']
xgbm = xgb.XGBRegressor(learning_rate =0.01, n_estimators=215, max_depth=10, min_child_weight=0.8, subsample=1, nthread=4)
rfe = RFE(xgbm, n_features_to_select= n_features_optimal)             
rfe = rfe.fit(x_train, y_train)

In [18]:
x_train_XGBM, x_test_XGBM = removeFeatures(x_train, x_test, rfe)

In [19]:
display('Featues chosen: ', set(x_train_XGBM.columns))
display('Features discarted: ',set(x_train.columns) - set(x_train_XGBM.columns))

In [20]:
xgbm = xgb.XGBRegressor(learning_rate =0.01, n_estimators=215, max_depth=10, min_child_weight=0.8, subsample=1, nthread=4)
start_time = time.time()
xgbm.fit(x_train_XGBM, y_train)
timeexecution['xgboost'] = (time.time() - start_time)
y_pred = xgbm.predict(x_test_XGBM)

In [21]:
#results = y_test.copy()
results['xgboost'] = y_pred

In [54]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2:', metrics.r2_score(y_test, y_pred))
mseresults['xgboost'] = metrics.mean_squared_error(y_test, y_pred)

Firstly, I create an explainer for the model, the inputs are the model, x_train and y_train. After, i can get the variable importance and the reverse cumulative distribution of residuals

In [28]:
variable_groups = {
    'Date': ['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA', 'TEMPORADA_COMERCIAL_ID'],
    'Product': ['PRODUCTO_ID', 'TALLA', 'GENERO_PRODUCTO', 'CATEGORIA', 'TIPOLOGIA', 'CONSUMER_COLOR', 'CREMALLERA', 'CORDONES', 'OUTSOLE_SUELA_TIPO', 'OUTSOLE_SUELA_SUBTIPO', 'PLANTILLA_EXTRAIBLE'],
    'Age': ['EDAD_COMPRA'],
    'ContanctInfo': ['NUMERO_DEUDOR_PAIS_ID', 'CIUDAD_CONTACTO', 'IDIOMA_CONTACTO'],
    'SalePerson': ['ESFUERZO_VENTA_ID']
}

In [29]:
xgboost_explainer = dx.Explainer(xgbm, x_test_XGBM, y_test)
explanation = xgboost_explainer.model_parts()
explanation.plot()
res = xgboost_explainer.model_performance(model_type='regression')
res.plot()
groupedexpl = xgboost_explainer.model_parts(variable_groups=variable_groups)
groupedexpl.plot()

In [30]:
print(xgboost_explainer.model_performance().result)

Also, I can group the features into a new ones. In this case, I have created different variables that include a similar meaning.
- Date: ANO_FACTURA, MES_FACTURA, FECHA_FACTURA, TEMPORADA_COMERCIAL_ID
- Product: PRODUCTO_ID, TALLA, GRUPO_ARTICULO_PRODUCTO_ID, GENERO_PRODUCTO, CATEGORIA, TIPOLOGIA, CONSUMER_COLOR, CREMALLERA, CORDONES, OUTSOLE_SUELA_TIPO, OUTSOLE_SUELA_SUBTIPO, PLANTILLA_EXTRAIBLE
- Age: EDAD_SN, EDAD_COMPRA
- ContanctInfo: NUMERO_DEUDOR_PAIS_ID, CIUDAD_CONTACTO, IDIOMA_CONTACTO, GENERO_CONTACTO, CONTACTO_SN
- SalePerson: ESFUERZO_VENTA_ID

In [33]:
xgb_sh_es_0 = xgboost_explainer.predict_parts(np.ravel(x_test_XGBM.loc[[indexspaindata[0]]]), type = 'shap')
xgb_sh_es_0.plot()
xgb_sh_es_1 = xgboost_explainer.predict_parts(np.ravel(x_test_XGBM.loc[[indexspaindata[20]]]), type = 'shap')
xgb_sh_es_1.plot()
xgb_sh_es_2 = xgboost_explainer.predict_parts(np.ravel(x_test_XGBM.loc[[indexspaindata[200]]]), type = 'shap')
xgb_sh_es_2.plot()
xgb_sh_es_3 = xgboost_explainer.predict_parts(np.ravel(x_test_XGBM.loc[[indexspaindata[500]]]), type = 'shap')
xgb_sh_es_3.plot()

In [34]:
xgb_sh_us_0 = xgboost_explainer.predict_parts(np.ravel(x_test_XGBM.loc[[indexusadata[0]]]), type = 'shap')
xgb_sh_us_0.plot()
xgb_sh_us_1 = xgboost_explainer.predict_parts(np.ravel(x_test_XGBM.loc[[indexusadata[20]]]), type = 'shap')
xgb_sh_us_1.plot()
xgb_sh_us_2 = xgboost_explainer.predict_parts(np.ravel(x_test_XGBM.loc[[indexusadata[200]]]), type = 'shap')
xgb_sh_us_2.plot()
xgb_sh_us_3 = xgboost_explainer.predict_parts(np.ravel(x_test_XGBM.loc[[indexusadata[500]]]), type = 'shap')
xgb_sh_us_3.plot()

## k-nearest neighbour regression

In [ ]:
rmse_val = [] #to store rmse values for different k
mse_val = []
model = neighbors.KNeighborsRegressor(n_neighbors = 7)
start_time = time.time()
model.fit(x_train, y_train)  #fit the model
timeexecution['knn'] = (time.time() - start_time)
y_pred=model.predict(x_test) #make prediction on test set
mse = metrics.mean_squared_error(y_test, y_pred)
error = np.sqrt(metrics.mean_squared_error(y_test, y_pred)) #calculate rmse
rmse_val.append(error) #store rmse values
mse_val.append(mse)
results['knn'] = y_pred
mseresults['knn'] = metrics.mean_squared_error(y_test, y_pred)

In [ ]:
#curvermse = pd.DataFrame(rmse_val) #elbow curve 
#curvermse.plot()
#curvemse = pd.DataFrame(mse_val)
#curvemse.plot()

## NN

### MLP Regressor

In [22]:
nn_reg = MLPRegressor(hidden_layer_sizes=(300, 300),  activation='logistic', solver='adam', alpha=0.01, batch_size='auto', learning_rate='constant', learning_rate_init=0.01, max_iter=1000, shuffle=False, tol=0.0001, verbose=True, early_stopping= True, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
nn_reg.fit(x_train, np.ravel(y_train))
y_pred = nn_reg.predict(x_test)
nn_reg.score(x_test, y_test)

In [23]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2:', metrics.r2_score(y_test, y_pred))

In [24]:
nn_reg_pred = lambda x: nn_reg.predict(x).astype(float)

In [25]:
results['nn'] = y_pred

In [130]:
indexspaindata = getIndexFilter(testdataset, 14, 'NUMERO_DEUDOR_PAIS_ID')
indexspaindata_original = getIndexFilter(testdataset_original, 'ES', 'NUMERO_DEUDOR_PAIS_ID')
print(indexspaindata)
print(indexspaindata_original)

In [123]:
#display(testdataset.loc[indexspaindata[0]])
#display(testdataset_original.loc[indexspaindata_original[0]])
#display(testdataset_original[(testdataset_original['PRODUCTO_ID'] == 10189) & (testdataset_original['NUMERO_DEUDOR_PAIS_ID'] == 'ES') & (test)].index)
#display(testdataset_original.loc[[644943]])
#i=0
#for v in indexspaindata_original:
#    if v == 644943:
#        break
#    else:
#        i = i+1
#print(i)


In [101]:
explainer = lime.lime_tabular.LimeTabularExplainer(x_train.values, feature_names=x_train.columns, verbose=True, mode='regression')
exp = explainer.explain_instance(np.ravel(x_test.iloc[[indexspaindata[0]]]), nn_reg_pred, num_features=10)
exp.show_in_notebook(show_table=True)
exp.as_pyplot_figure()

In [102]:
exp.as_list()

In [68]:
display(testdataset.iloc[[100]])
display(y_pred[100])

### LSTM

In [67]:
torch.manual_seed(39931191)
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.dropout = nn.Dropout(p= 0.1) # Dropout
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True) #lstm

        self.fc_1 =  nn.Linear(hidden_size, hidden_size//2) #fully connected 1
        self.fc = nn.Linear(hidden_size//2, 1) #fully connected last layer
        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        _, (hn, cn) = self.lstm(x, (h_0.detach(), c_0.detach())) #lstm with input, hidden, and internal state
        hn_fs = hn.view(self.num_layers, x.size(0), self.hidden_size)[-1] #reshaping the data for Dense layer next
        #out = self.dropout(hn_fs) # Dropout
        out = self.fc_1(hn_fs) # Dense
        out = self.relu(out) # Activation
        out = self.fc(out) # Dense
        out = self.relu(out) # Activation
        return out

In [68]:
num_epochs =  50 #1000 epochs
learning_rate = 0.01 #0.001 lr

input_size = 25 #number of features
hidden_size = 175 #number of features in hidden state
num_layers = 2 #number of stacked lstm layers

x_train_tensors = Variable(torch.Tensor(x_train.values))
x_test_tensors = Variable(torch.Tensor(x_test.values))
y_train_tensors = Variable(torch.Tensor(y_train.values))
y_test_tensors = Variable(torch.Tensor(y_test.values))

x_train_tensors_final = torch.reshape(x_train_tensors,   (x_train_tensors.shape[0], 1, x_train_tensors.shape[1]))
x_test_tensors_final = torch.reshape(x_test_tensors,  (x_test_tensors.shape[0], 1, x_test_tensors.shape[1]))

x_train_loader = DataLoader(x_train_tensors, batch_size= 200, shuffle= False, num_workers = 2)
x_test_loader = DataLoader(x_test_tensors, batch_size= 200, shuffle= False, num_workers = 2)
y_train_loader = DataLoader(y_train_tensors, batch_size= 200, shuffle= False, num_workers = 2)
y_test_loader = DataLoader(y_test_tensors, batch_size= 200, shuffle= False, num_workers = 2)

In [69]:
lstm_model = LSTM(input_size, hidden_size, num_layers) #our lstm class

In [70]:
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [71]:
min_val = sys.maxsize
count_epoch = 0
for epoch in range(num_epochs):
  for dtx, dty in zip(enumerate(x_train_loader), enumerate(y_train_loader)):
    xtr = torch.reshape(Variable(dtx[1]), (dtx[1].shape[0], 1, dtx[1].shape[1])) # Reshape
    outputs = lstm_model.forward(xtr) #forward pass
    optimizer.zero_grad() #caluclate the gradient, manually setting to 0
    loss = criterion(outputs, dty[1]) # obtain the loss function
    loss.backward() #calculates the loss of the loss function
    optimizer.step() #improve from loss, i.e backprop
  print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
  if (min_val - (min_val * 0.001)) > loss.item():
    min_val = loss.item()
    count_epoch = 0
  else:
    count_epoch = count_epoch + 1
  if count_epoch == 5:
    print('Model does not improve')
    #break

In [72]:
train_predict = lstm_model(x_test_tensors_final)#forward pass
data_predict = train_predict.data.numpy() #numpy conversion

In [73]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, data_predict))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, data_predict))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, data_predict)))
print('R2:', metrics.r2_score(y_test, data_predict))

In [74]:
results['lstm'] = data_predict

In [ ]:
def predictplot(dso, dsp, dso_size, range_plot):
    plt.axvline(x=dso_size, c='r', linestyle='--') #size of the training set
    plt.scatter(range(0, range_plot),y_test[:range_plot], label='Actual Data') #actual plot
    plt.scatter(range(0, range_plot), data_predict[:range_plot], label='Predicted Data') #predicted plot
    plt.title('Time-Series Prediction')
    plt.legend()
    plt.savefig('../Output/timeseriespredictionLSTM.png')
    plt.show()

In [ ]:
predictplot(y_test, data_predict, 5, 10)

In [55]:
indexspaindata = getIndexFilter(testdataset, 14, 'NUMERO_DEUDOR_PAIS_ID')
indexspaindata_original = getIndexFilter(testdataset_original, 'ES', 'NUMERO_DEUDOR_PAIS_ID')
print(indexspaindata)
print(indexspaindata_original)

In [148]:
print(testdataset_original.loc[[indexspaindata_original[0]]])
f = testdataset_original.sort_values(['PRODUCTO_ID'])
f = f.groupby(['PRODUCTO_ID', 'CONSUMER_COLOR']).IMP_VENTA_NETO_EUR.sum()
display(f.head(20))
data_info_original['CONSUMER_COLOR'].value_counts()

In [189]:
f = testdataset_original[testdataset_original.PRODUCTO_ID.str.contains('16002')].head(20)

In [67]:
display(testdataset.loc[[indexspaindata[0]]].values)

In [85]:
x_test.loc[[indexspaindata[20]]].values

In [108]:
print(testdataset_original.loc[[indexspaindata_original[0]]].values)
print(testdataset_original.loc[[indexspaindata_original[1]]].values)

In [112]:
ig = IntegratedGradients(lstm_model)
t = torch.Tensor(x_test.loc[[indexspaindata[0]]].values)
x_test_tensors_final = torch.reshape(t,  (t.shape[0], 1, t.shape[1]))
x_test_tensors_final_ig = torch.reshape(x_test_tensors_final[0],  (1, 1, 25))
attribution = ig.attribute(x_test_tensors_final_ig)
display(attribution.detach().numpy()[0][0])

In [110]:
def visualize_importance(title, features, importance, xtitle):
    plt.barh(range(0, features.size), importance, align = 'center')
    plt.yticks(range(0, features.size), features, wrap = True, fontsize = 7)
    plt.ylabel(xtitle)
    plt.title(title)
    plt.show()

In [111]:
visualize_importance('Variable Importance', x_test.columns, attribution.detach().numpy()[0][0], 'Feature')

In [178]:
def visualize_cummulative_importances(layer_1, layer_2, title="Average Feature Importances", plot=True, axis_title="Features"):
    cum_ly_1 = 0
    for i in range(len(layer_1)):
        cum_ly_1 = cum_ly_1 + layer_1[i]
    cum_ly_2 = 0
    for i in range((len(layer_2))):
        cum_ly_2 = cum_ly_2 + layer_2[i]
    print(title)
    print('Layer 1: ', cum_ly_1)
    print('Layer 2: ', cum_ly_2)
    if plot:
        plt.figure(figsize=(12,6))
        plt.bar(range(2), [cum_ly_1, cum_ly_2], align='center', width = [0.2, 0.2])
        plt.xticks(range(2), ['Layer 1', 'Layer 2'], wrap=True, fontsize = 10)
        plt.xlabel(axis_title)
        plt.title(title)

In [183]:
t = torch.Tensor(x_test.loc[[indexspaindata[0]]].values)
x_test_tensors_final = torch.reshape(t,  (t.shape[0], 1, t.shape[1]))
x_test_tensors_final_lc = torch.reshape(x_test_tensors_final[0],  (1, 1, 25))

layer_1 = LayerConductance(lstm_model, lstm_model.fc_1)
layer_1_vals = layer_1.attribute(x_test_tensors_final_lc)
layer_1_vals = (layer_1_vals.detach().numpy())

layer_2 = LayerConductance(lstm_model, lstm_model.fc)
layer_2_vals = layer_2.attribute(x_test_tensors_final_lc)
layer_2_vals = (layer_2_vals.detach().numpy())

visualize_cummulative_importances(layer_1_vals[0], layer_2_vals, title="Cumulative Neuron Importances", axis_title="Neurons")

In [186]:
np.mean(layer_2_vals)
np.mean(layer_1_vals)

## Prediction of intervals

As before, the base model is learnt from the training data. Then, a second model (the error model) is trained on a validation set to predict the squared difference between the predictions and the real values.

In [76]:
def takeVal(dt, ind, drange):
    money = []
    dayslist = []
    counter = []
    day = drange
    i = 0
    dayslist.append(str(dt.loc[[ind[(len(ind) - 1) - i]]].values[0][2]) + '/' + str(dt.loc[[ind[(len(ind) - 1) - i]]].values[0][1]) + '/' + str(dt.loc[[ind[(len(ind) - 1) - i]]].values[0][0]))
    while True:
        dayformatted = str(dt.loc[[ind[(len(ind) - 1) - i]]].values[0][2]) + '/' + str(dt.loc[[ind[(len(ind) - 1) - i]]].values[0][1]) + '/' + str(dt.loc[[ind[(len(ind) - 1) - i]]].values[0][0])
        if dayformatted not in dayslist:
            dayslist.append(dayformatted)
            day = day - 1
            counter.append(i)
        if day == 0:
            dayslist = dayslist[:drange]
            break
        i = i + 1
    return dayslist, counter
def sumPredictions(predictions, indexes):
    sum = 0
    sumpred = []
    aux = 0
    for p in predictions.values:
        sum = sum + float(p)
        if aux in indexes:
            sumpred.append(round(sum, 2))
            sum = 0
        aux = aux + 1
    return sumpred

In [77]:
print(x_train.shape)
print(x_test.shape)


In [78]:
indexspaindata = getIndexFilter(testdataset, 14, 'NUMERO_DEUDOR_PAIS_ID')
indexspaindata_original = getIndexFilter(testdataset_original, 'ES', 'NUMERO_DEUDOR_PAIS_ID')
usa_iso = getISOCountry('US')
indexusadata = getIndexFilter(testdataset, usa_iso, 'NUMERO_DEUDOR_PAIS_ID')
indexusadata_original = getIndexFilter(testdataset_original, 'US', 'NUMERO_DEUDOR_PAIS_ID')
ita_iso = getISOCountry('IT')
indexitadata = getIndexFilter(testdataset, ita_iso, 'NUMERO_DEUDOR_PAIS_ID')
indexitadata_original = getIndexFilter(testdataset_original, 'IT', 'NUMERO_DEUDOR_PAIS_ID')

In [79]:
SpaintakeVal, indexcountersp = takeVal(testdataset_original, indexspaindata_original, 20)
UsatakeVal, indexcounterus = takeVal(testdataset_original, indexusadata_original, 20)
ItatakeVal, indexcounterit = takeVal(testdataset_original, indexitadata_original, 20)

In [80]:
impsp = sumPredictions(results[results.index.isin(indexspaindata)]['Import'], indexcountersp)
impus = sumPredictions(results[results.index.isin(indexusadata)]['Import'], indexcounterus)
impita = sumPredictions(results[results.index.isin(indexitadata)]['Import'], indexcounterit)

In [30]:
predsp = sumPredictions(results[results.index.isin(indexspaindata)]['lm'], indexcountersp)
predus = sumPredictions(results[results.index.isin(indexusadata)]['lm'], indexcounterus)
predita = sumPredictions(results[results.index.isin(indexitadata)]['lm'], indexcounterit)

In [31]:
st_dev_lm_sp = round(metrics.mean_squared_error(impsp, predsp) ** 0.5, 2)
upper = impsp + st_dev_lm_sp
lower = impsp - st_dev_lm_sp

In [32]:
plt.plot(SpaintakeVal, impsp, '.', color = 'k')
plt.plot(SpaintakeVal, predsp, '.', color = 'g')
plt.fill_between(SpaintakeVal, upper, lower, alpha=0.2)
plt.xticks(SpaintakeVal, rotation=85)
plt.title('Prediction Interval (€) - Spain')
plt.legend(['Real Import', 'Prediction', 'Prediction Interval'])
plt.show()

In [33]:
st_dev_lm_us = round(metrics.mean_squared_error(impus, predus) ** 0.5, 2)

In [34]:
plt.plot(UsatakeVal, impus, '.', color =  'k')
plt.plot(UsatakeVal, predus, '.', color =  'g')
plt.fill_between(UsatakeVal, (impus + st_dev_lm_us), (impus - st_dev_lm_us), alpha=0.2)
plt.xticks(UsatakeVal, rotation=85)
plt.title('Prediction Interval (€) - US')
plt.legend(['Real Import', 'Prediction', 'Prediction Interval'])
plt.show()

In [35]:
st_dev_lm_ita = round(metrics.mean_squared_error(impita, predita) ** 0.5, 2)

In [36]:
plt.plot(ItatakeVal, impita, '.', color =  'k')
plt.plot(ItatakeVal, predita, '.', color =  'g')
plt.fill_between(ItatakeVal, (impita + st_dev_lm_ita), (impita - st_dev_lm_ita), alpha=0.2)
plt.xticks(ItatakeVal, rotation=85)
plt.title('Prediction Interval (€) - Italy')
plt.legend(['Real Import', 'Prediction', 'Prediction Interval'])
plt.show()

In [37]:
print('Spain: ', st_dev_lm_sp)
print('US: ', st_dev_lm_us)
print('Italy: ', st_dev_lm_ita)

In [46]:
predsp_xgboost = sumPredictions(results[results.index.isin(indexspaindata)]['xgboost'], indexcountersp)
predus_xgboost = sumPredictions(results[results.index.isin(indexusadata)]['xgboost'], indexcounterus)
predita_xgboost = sumPredictions(results[results.index.isin(indexitadata)]['xgboost'], indexcounterit)

In [47]:
st_dev_xgboost_sp = round(metrics.mean_squared_error(impsp, predsp_xgboost) ** 0.5, 2)

In [48]:
plt.plot(SpaintakeVal, impsp, '.', color =  'k')
plt.plot(SpaintakeVal, predsp_xgboost, '.', color =  'g')
plt.fill_between(range(len(SpaintakeVal)), impsp + st_dev_xgboost_sp, impsp - st_dev_xgboost_sp, alpha=0.2)
plt.xticks(SpaintakeVal, rotation=85)
plt.title('Prediction Interval (€) - Spain')
plt.legend(['Real Import', 'Prediction', 'Prediction Interval'])
plt.show()

In [49]:
st_dev_xgboost_us = round(metrics.mean_squared_error(impus, predus_xgboost) ** 0.5, 2)

In [50]:
plt.plot(UsatakeVal, impus, '.', color = 'k')
plt.plot(UsatakeVal, predus_xgboost, '.', color = 'g')
plt.fill_between(range(len(UsatakeVal)), (impus + st_dev_xgboost_us), (impus - st_dev_xgboost_us), alpha=0.2)
plt.xticks(UsatakeVal, rotation=85)
plt.title('Prediction Interval (€) - US')
plt.legend(['Real Import', 'Prediction', 'Prediction Interval'])
plt.show()

In [51]:
st_dev_xgboost_ita = round(metrics.mean_squared_error(impita, predita_xgboost) ** 0.5, 2)

In [52]:
plt.plot(ItatakeVal, impita, '.', color = 'k')
plt.plot(ItatakeVal, predita_xgboost, '.', color = 'g')
plt.fill_between(range(len(ItatakeVal)), (impita + st_dev_xgboost_ita), (impita - st_dev_xgboost_ita), alpha=0.2)
plt.xticks(ItatakeVal, rotation=85)
plt.title('Prediction Interval (€) - Italy')
plt.legend(['Real Import', 'Prediction', 'Prediction Interval'])
plt.show()

In [53]:
print('Spain: ', st_dev_xgboost_sp)
print('US: ', st_dev_xgboost_us)
print('Italy: ', st_dev_xgboost_ita)

In [54]:
predsp_nn = sumPredictions(results[results.index.isin(indexspaindata)]['nn'], indexcountersp)
predus_nn = sumPredictions(results[results.index.isin(indexusadata)]['nn'], indexcounterus)
predita_nn = sumPredictions(results[results.index.isin(indexitadata)]['nn'], indexcounterit)

In [55]:
st_dev_nn_sp = round(metrics.mean_squared_error(impsp, predsp_nn) ** 0.5, 2)

In [56]:
plt.plot(SpaintakeVal, impsp, '.', color =  'k')
plt.plot(SpaintakeVal, predsp_nn, '.', color =  'g')
plt.fill_between(SpaintakeVal, impsp + st_dev_nn_sp, impsp - st_dev_nn_sp, alpha=0.2)
plt.xticks(SpaintakeVal, rotation=85)
plt.title('Prediction Interval (€) - Spain')
plt.legend(['Real Import', 'Prediction', 'Prediction Interval'])
plt.show()

In [58]:
st_dev_nn_us = round(metrics.mean_squared_error(impus, predus_nn) ** 0.5, 2)

In [59]:
plt.plot(UsatakeVal, impus, '.', color = 'k')
plt.plot(UsatakeVal, predus_nn, '.', color = 'g')
plt.fill_between(UsatakeVal, (impus + st_dev_nn_us), (impus - st_dev_nn_us), alpha=0.2)
plt.xticks(UsatakeVal, rotation=85)
plt.title('Prediction Interval (€) - US')
plt.legend(['Real Import', 'Prediction', 'Prediction Interval'])
plt.show()

In [62]:
st_dev_nn_ita = round(metrics.mean_squared_error(impita, predita_nn) ** 0.5, 2)

In [63]:
plt.plot(ItatakeVal, impita, '.', color = 'k')
plt.plot(ItatakeVal, predita_nn, '.', color = 'g')
plt.fill_between(ItatakeVal, (impita + st_dev_nn_ita), (impita - st_dev_nn_ita), alpha=0.2)
plt.xticks(ItatakeVal, rotation=85)
plt.title('Prediction Interval (€) - Italy')
plt.legend(['Real Import', 'Prediction', 'Prediction Interval'])
plt.show()

In [64]:
print('Spain: ', st_dev_nn_sp)
print('US: ', st_dev_nn_us)
print('Italy: ', st_dev_nn_ita)

In [82]:
predsp_lstm = sumPredictions(results[results.index.isin(indexspaindata)]['lstm'], indexcountersp)
predus_lstm = sumPredictions(results[results.index.isin(indexusadata)]['lstm'], indexcounterus)
predita_lstm = sumPredictions(results[results.index.isin(indexitadata)]['lstm'], indexcounterit)

In [83]:
st_dev_lstm_sp = round(metrics.mean_squared_error(impsp, predsp_lstm) ** 0.5, 2)

In [84]:
plt.plot(SpaintakeVal, impsp, '.', color =  'k')
plt.plot(SpaintakeVal, predsp_lstm, '.', color =  'g')
plt.fill_between(SpaintakeVal, impsp + st_dev_lstm_sp, impsp - st_dev_lstm_sp, alpha=0.2)
plt.xticks(SpaintakeVal, rotation=85)
plt.title('Prediction Interval (€) - Spain')
plt.legend(['Real Import', 'Prediction', 'Prediction Interval'])
plt.show()

In [85]:
st_dev_lstm_us = round(metrics.mean_squared_error(impus, predus_lstm) ** 0.5, 2)

In [86]:
plt.plot(UsatakeVal, impus, '.', color = 'k')
plt.plot(UsatakeVal, predus_lstm, '.', color = 'g')
plt.fill_between(UsatakeVal, (impus + st_dev_lstm_us), (impus - st_dev_lstm_us), alpha=0.2)
plt.xticks(UsatakeVal, rotation=85)
plt.title('Prediction Interval (€) - US')
plt.legend(['Real Import', 'Prediction', 'Prediction Interval'])
plt.show()

In [87]:
st_dev_lstm_ita = round(metrics.mean_squared_error(impita, predita_lstm) ** 0.5, 2)

In [88]:
plt.plot(ItatakeVal, impita, '.', color = 'k')
plt.plot(ItatakeVal, predita_lstm, '.', color = 'g')
plt.fill_between(ItatakeVal, (impita + st_dev_lstm_ita), (impita - st_dev_lstm_ita), alpha=0.2)
plt.xticks(ItatakeVal, rotation=85)
plt.title('Prediction Interval (€) - Italy')
plt.legend(['Real Import', 'Prediction', 'Prediction Interval'])
plt.show()